Created on Friday 08 January 2021

**Group 2 - Identification**

**"ZDNet" scraper**

@authors: Marianne Manson

Liens utilisés

https://www.zdnet.fr/actualites/la-data-est-notre-or-noir-mais-quel-est-son-moteur-39881697.htm

Listes:

- https://www.zdnet.fr/actualites/gestion-de-donnees-4000237197q.htm
- https://www.zdnet.fr/actualites/cybersecurite-3900046206q.htm
- https://www.zdnet.fr/actualites/logiciels-3900046188q.htm
- https://www.zdnet.fr/actualites/transformation-numerique-4000237526q.htm


In [5]:
#import de lib
import pandas as pd
import numpy as np
import re
from datetime import datetime
from requests import get
from bs4 import BeautifulSoup

In [12]:
df_scrap = pd.DataFrame(columns=['art_content','art_content_html','art_extract_datetime','art_lang','art_title','art_url','src_name','src_type','src_url','src_img','art_auth','art_tag']  )
df_scrap

,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag


In [17]:
def get_url_article_zdnet(page_url):
    """Documentation
    
    Parameters:
        page_url: url of the page where articles are listed
    
    Out:
        url_art_list: list with the urls of articles
    """
    req = get(page_url)
    html_soup = BeautifulSoup(req.text, 'html.parser')
    zones_liste = html_soup.find_all("article",{'class':"item"})
    url_art_list = []
    for zone in zones_liste:
        title_zone = zone.find("h3")
        url_article = title_zone.find("a")['href']
        url_art_list.append("https://www.zdnet.fr" + url_article)
    return url_art_list

In [18]:
test_list = get_url_article_zdnet("https://www.zdnet.fr/actualites/cybersecurite-3900046206q.htm")
test_list

['https://www.zdnet.fr/actualites/chiffres-cles-le-marche-des-logiciels-de-securite-informatique-39790928.htm',
 'https://www.zdnet.fr/guide-achat/les-meilleurs-antivirus-pour-vos-pc-smartphones-et-tablettes-39909005.htm',
 'https://www.zdnet.fr/guide-achat/les-meilleurs-vpn-pour-le-teletravail-et-les-connexions-a-distance-39908115.htm',
 'https://www.zdnet.fr/guide-achat/les-meilleures-cles-d-authentification-multifacteurs-en-2020-39900941.htm',
 'https://www.zdnet.fr/guide-achat/google-authenticator-voici-pourquoi-vous-devriez-vous-en-debarrasser-39897597.htm',
 'https://www.zdnet.fr/actualites/la-gestion-des-acces-a-privileges-une-priorite-dans-la-cybersecurite-des-villes-39915735.htm',
 'https://www.zdnet.fr/actualites/ransomware-ryuk-aurait-empoche-plus-de-150-millions-de-dollars-39915797.htm',
 'https://www.zdnet.fr/actualites/solarwinds-la-societe-jetbrains-nie-etre-melee-au-piratage-39915749.htm',
 'https://www.zdnet.fr/actualites/ia-et-cybersecurite-une-association-gagnante-39

In [26]:
def scraping(url):
    '''Documentation
    
    Parameters:
        url: url of the scraped page
    
    Out:
        new_row: data to put in dataframe
    '''
    req = get(url)
    html_soup = BeautifulSoup(req.text, 'html.parser')
    # content, html_content
    content_html = html_soup.find("div",{'class':"storyBody"})
    content = content_html.text
    # date, author
    zone_infos = html_soup.find("div",{'class':"byline"})
    zone_infos = zone_infos.find("p",{'class':"meta"})
    ## author
    try:
        zone_author = zone_infos.find("span")
        author = zone_author.find("span").text
    except:
        author = "no_data"
    ## date
    date = zone_infos.find("time")['datetime']
    format_end = date[-5:]
    date = datetime.strptime(date,'%Y-%m-%dT%H:%M:%S+'+format_end)
    date = date.strftime('%Y-%m-%d')
    # title
    title = html_soup.find("h1").text
    # img
    try:
        img = content_html.find("img")['src']
    except:
        img = "no_data"
    #tags
    zone_tags = html_soup.find("p",{'class':"relatedTopics"})
    list_tags_links = zone_tags.find_all("a")
    list_tags = []
    for link in list_tags_links:
        list_tags.append(link.text)
    # data to add in dataframe 
    new_row = {'art_content': content_html ,
               'art_content_html': content ,
               'art_extract_datetime': date ,
               'art_lang': 'fr' , 
               'art_title' : title , 
               'art_url' : url ,
               'src_name' :'zdnet',
               'src_type' : 'xpath_source',
               'src_url' : 'https://www.zdnet.fr/',
               'src_img' : img ,
               'art_auth': author ,
               'art_tag': list_tags}
    return new_row

In [27]:
new_row = scraping("https://www.zdnet.fr/actualites/la-data-est-notre-or-noir-mais-quel-est-son-moteur-39881697.htm")
df_test = df_scrap.append(new_row, ignore_index=True)
df_test

,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag
0,"[\n, [[], ], [« Data is the new oil » lit-on ...",\n « Data is the new oil » lit-on depuis un mo...,2019-03-14,fr,\n La data est notre « or noir » ? Mais...,https://www.zdnet.fr/actualites/la-data-est-no...,zdnet,xpath_source,https://www.zdnet.fr/,https://d1fmx1rbmqrxrr.cloudfront.net/zdnet/i/...,"Adrien Blind, Saagie","[Big data, Intelligence artificielle (IA), Dev..."


In [30]:
#test on a page
df_test = df_scrap
test_list = get_url_article_zdnet("https://www.zdnet.fr/actualites/transformation-numerique-4000237526q.htm")
for url_art in test_list:
    #print(url_art)
    new_row = scraping(url_art)
    df_test = df_test.append(new_row, ignore_index=True)
df_test

,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag
0,"[\n, [\n, [\n, <img alt=""Les meilleurs service...","\n\n\n\n\n\nEn 2007, Drew Houston, le PDG de D...",2020-09-28,fr,\n Les meilleurs services de stockage c...,https://www.zdnet.fr/guide-achat/les-meilleurs...,zdnet,xpath_source,https://www.zdnet.fr/,https://d1fmx1rbmqrxrr.cloudfront.net/zdnet/op...,Steven J. Vaughan-Nichols,"[Stockage, Cloud computing, Cloud public, SaaS..."
1,"[\n, [\n, [\n, <img alt=""Les meilleurs drones ...",\n\n\n\n\n\nLa photographie et le cinéma sont ...,2020-09-24,fr,\n Les meilleurs drones de photographie...,https://www.zdnet.fr/guide-achat/les-meilleurs...,zdnet,xpath_source,https://www.zdnet.fr/,https://d1fmx1rbmqrxrr.cloudfront.net/zdnet/op...,Greg Nichols,"[Drone, Transformation Numérique]"
2,"[\n, [\n, [\n, <img alt=""Les meilleurs robots ...",\n\n\n\n\n\nSommaire\nDouble 2 par Double Robo...,2020-09-03,fr,\n Les meilleurs robots de téléprésence...,https://www.zdnet.fr/guide-achat/les-meilleurs...,zdnet,xpath_source,https://www.zdnet.fr/,https://d1fmx1rbmqrxrr.cloudfront.net/zdnet/op...,Greg Nichols,"[Télécommunications, Télétravail, Transformati..."
3,"[\n, [\n, [\n, <img alt=""Notre sélection du me...",\n\n\n\n\n\nQui dit télétravail dit forcément ...,2020-07-02,fr,\n Notre sélection du meilleur matériel...,https://www.zdnet.fr/guide-achat/notre-selecti...,zdnet,xpath_source,https://www.zdnet.fr/,https://d1fmx1rbmqrxrr.cloudfront.net/zdnet/op...,La rédaction de ZDNet.fr,"[Guide d'achat, Poste de travail, Télétravail,..."
4,"[\n, [\n, [\n, <img alt=""Les Ray-Ban intellige...",\n\n\n\n\n\nL'équipe en charge de la réalité a...,2021-01-07,fr,\n Les Ray-Ban intelligentes de Faceboo...,https://www.zdnet.fr/actualites/les-ray-ban-in...,zdnet,xpath_source,https://www.zdnet.fr/,https://d1fmx1rbmqrxrr.cloudfront.net/zdnet/op...,Liam Tung,"[Facebook, Technologie, Objets connectés, Tran..."
5,"[\n, [\n, [\n, <img alt=""Orange Bank mise sur ...","\n\n\n\n\n\nDu côté de la direction d'Orange, ...",2021-01-07,fr,\n Orange Bank mise sur le B to B avec...,https://www.zdnet.fr/actualites/orange-bank-mi...,zdnet,xpath_source,https://www.zdnet.fr/,https://d1fmx1rbmqrxrr.cloudfront.net/zdnet/op...,La rédaction de ZDNet.fr,"[Orange, Banque Mobile, Transformation Numériq..."
6,"[\n, [\n, [\n, <img alt=""Faire entrer la canti...","\n\n\n\n\n\nImaginez que vous êtes au bureau, ...",2021-01-08,fr,\n Faire entrer la cantine connectée su...,https://www.zdnet.fr/actualites/faire-entrer-l...,zdnet,xpath_source,https://www.zdnet.fr/,https://d1fmx1rbmqrxrr.cloudfront.net/zdnet/op...,Clarisse Treilles,"[Transformation Numérique, Technologie, Télétr..."
7,"[\n, [\n, [\n, <img alt=""La revanche des QR co...",\n\n\n\n\n\n Image : Service NSW\nVous vous so...,2021-01-07,fr,\n La revanche des QR codes ?\n,https://www.zdnet.fr/actualites/la-revanche-de...,zdnet,xpath_source,https://www.zdnet.fr/,https://d1fmx1rbmqrxrr.cloudfront.net/zdnet/op...,Chris Duckett,"[Technologie, Transformation Numérique, Santé,..."
8,"[\n, [\n, [\n, <img alt=""Australie : Une étude...",\n\n\n\n\n\nUn groupe de chercheurs australien...,2021-01-06,fr,\n Australie : Une étude utilise les do...,https://www.zdnet.fr/actualites/australie-une-...,zdnet,xpath_source,https://www.zdnet.fr/,https://zdnet4.cbsistatic.com/hub/i/2019/07/24...,Aimee Chanthadavong,"[Santé, Transformation Numérique, Base de donn..."
9,"[\n, [\n, [\n, <img alt=""Teledyne achète les c...","\n\n\n\n\n\nFlir, créée en 1978, est une socié...",2021-01-06,fr,\n Teledyne achète les caméras thermiqu...,https://www.zdnet.fr/actualites/teledyne-achet...,zdnet,xpath_source,https://www.zdnet.fr/,https://d1fmx1rbmqrxrr.cloudfront.net/zdnet/op...,La rédaction de ZDNet.fr,"[Transformation Numérique, Technologie]"
